#### This section is to create Xgeo conda env

%conda create -n xgeo -c conda-forge install -y xgeo pandas==0.24.1 xarray==0.11.3 rasterio==1.0.18 geopandas==0.4.0 netCDF4==1.4.2 shapely==1.6.4 ipykernel scikit-learn

%conda init

%source .bashrc

%conda activate xgeo

%python -m ipykernel install --user --name Py3-XGeo

In [18]:
##

import logging
logger = logging.getLogger(__name__)
import os

import xarray as xr
import xgeo # Needs to be imported to use geo extension


from matplotlib import pyplot as plt
import gdal
# from sklearn import cluster
# from skimage import exposure
import numpy as np
#
#
# # from docopt import docopt
# from osgeo import gdal
# from sklearn import metrics
# import numpy as np
# import matplotlib.pyplot as plt
# from matplotlib.colors import ListedColormap
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.datasets import make_moons, make_circles, make_classification
# from sklearn.neural_network import MLPClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC
# from sklearn.gaussian_process import GaussianProcessClassifier
# from sklearn.gaussian_process.kernels import RBF
# from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
# from sklearn.naive_bayes import GaussianNB
# from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from pathlib import Path

import geopandas as gpd


# BASE_DIR = Path(__file__).resolve().parent.parent
BASE_DIR = Path().resolve()

Prj_Path = BASE_DIR.joinpath('./')# os.path.join(Shared, './rss_general_classifier/rss_general_classifier')
print(Prj_Path)

/home/jovyan/shared/Islam/rss_general_classifier/rss_general_classifier


In [2]:

# Prj_Path = os.chdir(os.path.join(Shared, './rss_general_classifier/rss_general_classifier')
raster_data_path = Prj_Path.joinpath('./data/image/crop_p224r63_all_bands_29192.tif') # os.path.join(Prj_Path, file_path)
output_fname = Prj_Path.joinpath('./data/image/crop_p224r63_all_bands_29192_train.tif')
method = 'random-forest'
train_data_path = Prj_Path.joinpath('./data/train/train_data_29192.shp')
validation_data_path = Prj_Path.joinpath('./data/test/val_data_29192.shp')


In [3]:
ds = xr.open_rasterio(raster_data_path)

dset= ds.to_dataset(name='image')
ds_masked = dset.where(dset.band >= ds.nodatavals[1])
ds_masked.geo.stats()
ds_masked


/opt/conda/envs/xgeo/lib/python3.7/site-packages/xgeo/raster.py:95: UserWarning: There is no time_dim dimension in the DataArray. It will be added to the dataarray.
  "There is no {0} dimension in the DataArray. It will be added to the dataarray.".format(dim)
/opt/conda/envs/xgeo/lib/python3.7/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


<xarray.Dataset>
Dimensions:  (band: 5, time: 1, x: 2212, y: 1170)
Coordinates:
  * band     (band) int64 1 2 3 4 5
  * y        (y) float64 9.562e+06 9.562e+06 9.562e+06 ... 9.529e+06 9.529e+06
  * x        (x) float64 5.942e+05 5.942e+05 5.943e+05 ... 6.572e+05 6.572e+05
Dimensions without coordinates: time
Data variables:
    image    (time, band, y, x) float32 -3.4e+38 -3.4e+38 ... -3.4e+38 -3.4e+38
Attributes:
    transform:    (28.49991227616556, 0, 594202.6568389062, 0, -28.4999122768...
    crs:          +init=epsg:29192
    bounds:       (594202.6568389062, 9528550.32978707, 657244.4627937844, 95...
    origin:       top_left
    resolutions:  (28.49991227616556, -28.499912276864052)

In [4]:
ds.nodatavals[1]

-3.4e+38

In [5]:
ds = ds.where(ds["band"] != ds.nodatavals[1])
ds

<xarray.DataArray (band: 5, y: 1170, x: 2212)>
array([[[-3.4e+38, -3.4e+38, ..., -3.4e+38, -3.4e+38],
        [-3.4e+38, -3.4e+38, ..., -3.4e+38, -3.4e+38],
        ...,
        [-3.4e+38, -3.4e+38, ..., -3.4e+38, -3.4e+38],
        [-3.4e+38, -3.4e+38, ..., -3.4e+38, -3.4e+38]],

       [[-3.4e+38, -3.4e+38, ..., -3.4e+38, -3.4e+38],
        [-3.4e+38, -3.4e+38, ..., -3.4e+38, -3.4e+38],
        ...,
        [-3.4e+38, -3.4e+38, ..., -3.4e+38, -3.4e+38],
        [-3.4e+38, -3.4e+38, ..., -3.4e+38, -3.4e+38]],

       ...,

       [[-3.4e+38, -3.4e+38, ..., -3.4e+38, -3.4e+38],
        [-3.4e+38, -3.4e+38, ..., -3.4e+38, -3.4e+38],
        ...,
        [-3.4e+38, -3.4e+38, ..., -3.4e+38, -3.4e+38],
        [-3.4e+38, -3.4e+38, ..., -3.4e+38, -3.4e+38]],

       [[-3.4e+38, -3.4e+38, ..., -3.4e+38, -3.4e+38],
        [-3.4e+38, -3.4e+38, ..., -3.4e+38, -3.4e+38],
        ...,
        [-3.4e+38, -3.4e+38, ..., -3.4e+38, -3.4e+38],
        [-3.4e+38, -3.4e+38, ..., -3.4e+38, -3.4e+38]]], 

In [6]:
##
print(ds)
# replace all band's values less than -9999 with np.nan
ds_masked = ds.where(ds.band < -9999)

<xarray.DataArray (band: 5, y: 1170, x: 2212)>
array([[[-3.4e+38, -3.4e+38, ..., -3.4e+38, -3.4e+38],
        [-3.4e+38, -3.4e+38, ..., -3.4e+38, -3.4e+38],
        ...,
        [-3.4e+38, -3.4e+38, ..., -3.4e+38, -3.4e+38],
        [-3.4e+38, -3.4e+38, ..., -3.4e+38, -3.4e+38]],

       [[-3.4e+38, -3.4e+38, ..., -3.4e+38, -3.4e+38],
        [-3.4e+38, -3.4e+38, ..., -3.4e+38, -3.4e+38],
        ...,
        [-3.4e+38, -3.4e+38, ..., -3.4e+38, -3.4e+38],
        [-3.4e+38, -3.4e+38, ..., -3.4e+38, -3.4e+38]],

       ...,

       [[-3.4e+38, -3.4e+38, ..., -3.4e+38, -3.4e+38],
        [-3.4e+38, -3.4e+38, ..., -3.4e+38, -3.4e+38],
        ...,
        [-3.4e+38, -3.4e+38, ..., -3.4e+38, -3.4e+38],
        [-3.4e+38, -3.4e+38, ..., -3.4e+38, -3.4e+38]],

       [[-3.4e+38, -3.4e+38, ..., -3.4e+38, -3.4e+38],
        [-3.4e+38, -3.4e+38, ..., -3.4e+38, -3.4e+38],
        ...,
        [-3.4e+38, -3.4e+38, ..., -3.4e+38, -3.4e+38],
        [-3.4e+38, -3.4e+38, ..., -3.4e+38, -3.4e+38]]], 

In [7]:
##
print(ds_masked)
dset= ds_masked.to_dataset(name='image')
dset.geo.stats()

<xarray.DataArray (band: 5, y: 1170, x: 2212)>
array([[[nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan],
        ...,
        [nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan]],

       [[nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan],
        ...,
        [nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan]],

       ...,

       [[nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan],
        ...,
        [nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan]],

       [[nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan],
        ...,
        [nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan]]], dtype=float32)
Coordinates:
  * band     (band) int64 1 2 3 4 5
  * y        (y) float64 9.562e+06 9.562e+06 9.562e+06 ... 9.529e+06 9.529e+06
  * x        (x) float64 5.942e+05 5.942e+05 5.943e+05 ... 6.572e+05 6.572e+05
Attributes:
    transform:   (28.49991227626653, 0.0, 594202.6568389062, 0.0, -28.4999122...
    c

/opt/conda/envs/xgeo/lib/python3.7/site-packages/xgeo/raster.py:95: UserWarning: There is no time_dim dimension in the DataArray. It will be added to the dataarray.
  "There is no {0} dimension in the DataArray. It will be added to the dataarray.".format(dim)
/opt/conda/envs/xgeo/lib/python3.7/site-packages/xarray/core/nanops.py:161: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/opt/conda/envs/xgeo/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1628: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/opt/conda/envs/xgeo/lib/python3.7/site-packages/xarray/core/nputils.py:222: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwds)


,,image_mean,image_std,image_min,image_max
band,time,,,,
1,0,NaN,NaN,NaN,NaN
2,0,NaN,NaN,NaN,NaN
3,0,NaN,NaN,NaN,NaN
4,0,NaN,NaN,NaN,NaN
5,0,NaN,NaN,NaN,NaN


In [8]:
##general_classifier/data/train/train_data_29192.shp
train_data_file = Prj_Path.joinpath('./data/train/train_data_29192.shp')
shp = gpd.read_file(train_data_file)
shp["class_id"] = shp["class_id"].astype(int)
df = dset.to_dataframe()
sampled = dset.geo.sample(vector_file=shp, value_name='class_id')

In [9]:
df_sampled = sampled.unstack()
features = df_sampled.reset_index()
features.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 4 columns):
(class_id, )    0 non-null float64
(x, )           0 non-null float64
(y, )           0 non-null float64
(time, )        0 non-null float64
dtypes: float64(4)
memory usage: 76.0 bytes


features.iloc[:,[4,5,6,7,8]]
##
X = features.iloc[:,[x for x in range(4,9)]]
Y = features.iloc[:,[x for x in range(0,1)]]

##
from sklearn.model_selection import train_test_split
X_train, X_test,Y_train, Y_test = train_test_split(X, Y, test_size=0.4)

In [78]:

import xarray as xr
import xgeo # Needs to be imported to use geo extension

import geopandas as gpd
import gdal

from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

import logging
logger = logging.getLogger(__name__)

# A list of "random" colors
COLORS = [
    "#000000", "#FFFF00", "#1CE6FF", "#FF34FF", "#FF4A46", "#008941", "#006FA6", "#A30059",
    "#FFDBE5", "#7A4900", "#0000A6", "#63FFAC", "#B79762", "#004D43", "#8FB0FF", "#997D87",
    "#5A0007", "#809693", "#FEFFE6", "#1B4400", "#4FC601", "#3B5DFF", "#4A3B53", "#FF2F80",
    "#61615A", "#BA0900", "#6B7900", "#00C2A0", "#FFAA92", "#FF90C9", "#B903AA", "#D16100",
    "#DDEFFF", "#000035", "#7B4F4B", "#A1C299", "#300018", "#0AA6D8", "#013349", "#00846F",
    "#372101", "#FFB500", "#C2FFED", "#A079BF", "#CC0744", "#C0B9B2", "#C2FF99", "#001E09",
    "#00489C", "#6F0062", "#0CBD66", "#EEC3FF", "#456D75", "#B77B68", "#7A87A1", "#788D66",
    "#885578", "#FAD09F", "#FF8A9A", "#D157A0", "#BEC459", "#456648", "#0086ED", "#886F4C",
    "#34362D", "#B4A8BD", "#00A6AA", "#452C2C", "#636375", "#A3C8C9", "#FF913F", "#938A81",
    "#575329", "#00FECF", "#B05B6F", "#8CD0FF", "#3B9700", "#04F757", "#C8A1A1", "#1E6E00",
    "#7900D7", "#A77500", "#6367A9", "#A05837", "#6B002C", "#772600", "#D790FF", "#9B9700",
    "#549E79", "#FFF69F", "#201625", "#72418F", "#BC23FF", "#99ADC0", "#3A2465", "#922329",
    "#5B4534", "#FDE8DC", "#404E55", "#0089A3", "#CB7E98", "#A4E804", "#324E72", "#6A3A4C",
    "#83AB58", "#001C1E", "#D1F7CE", "#004B28", "#C8D0F6", "#A3A489", "#806C66", "#222800",
    "#BF5650", "#E83000", "#66796D", "#DA007C", "#FF1A59", "#8ADBB4", "#1E0200", "#5B4E51",
    "#C895C5", "#320033", "#FF6832", "#66E1D3", "#CFCDAC", "#D0AC94", "#7ED379", "#012C58"
]


def dataframe_to_features(dataset_geo):
    features = dataset_geo.reset_index()
    
    # This is more time efficient 
    #X = sampled_dataset_geo.iloc[:, 0:]
    #y = sampled_dataset_geo.iloc[:,[x for x in range(0,1)]]
    
    # Return sampled pixel and its label for traning
    if 'class_id' in features.columns:
        X = features.image
        y = features.class_id
        
        return X, y
    
    # Return only the pixel value of each band for predection
    else:
        X = features.image
        return X


    
def sample_vectors_to_raster(vector_file_path, dataset_geo):
    """
    Rasterize, in a dataset, the vector file in the given path.
    The data of each file will be assigned the same pixel value. This value is defined by the order
    of the file in file_paths, starting with 1: so the points/poligons/etc in the same file will be
    marked as 1, those in the second file will be 2, and so on.
    :param vector_file_path: Path to a directory with shapefiles
    :param dataset_geo: Number of rows of the result
    :param cols: Number of columns of the result
    :param geo_transform: Returned value of gdal.Dataset.GetGeoTransform (coefficients for
                          transforming between pixel/line (P,L) raster space, and projection
                          coordinates (Xp,Yp) space.
    :param projection: Projection definition string (Returned by gdal.Dataset.GetProjectionRef)
    """
    
    # Read the shape file from the given path
    shp = gpd.read_file(vector_file_path)
    
    # This line to cast the string in the shape file to int so xgeo would not give an error 
    shp["class_id"] = shp["class_id"].astype(int)
    
    # Sample the dataset with the given vector file
    sampled_labeled_raster = dataset_geo.geo.sample(vector_file=shp, value_name='class_id')
    
    # This to unstack the dataframe to be suitable for features
    df_labeled_raster = sampled_labeled_raster.unstack()
    
    """
    ## The pervious line proivde the following outputs which is super handy
    df_sampled
                                             image                        
    band                                         1     2     3     4     5
    class_id x             y            time                              
    1.0      597038.398110 9.547916e+06 0     56.0  23.0  17.0  64.0  61.0
                           9.547945e+06 0     59.0  23.0  19.0  75.0  54.0
                                            ...   ...   ...   ...   ...
    3.0      654437.221435 9.536488e+06 0     54.0  17.0  10.0   3.0   1.0
                       9.536516e+06 0     54.0  17.0  12.0   3.0   1.0
                       9.536545e+06 0     51.0  17.0  11.0   5.0   1.0
                                            ...   ...   ...   ...   ...                       
    """

    return df_labeled_raster

def write_geotiff(fname, classes, data, geo_transform, projection, data_type=gdal.GDT_Byte):
    """
    Create a GeoTIFF file with the given data.
    :param fname: Path to a directory with shapefiles
    :param data: Number of rows of the result
    :param geo_transform: Returned value of gdal.Dataset.GetGeoTransform (coefficients for
                          transforming between pixel/line (P,L) raster space, and projection
                          coordinates (Xp,Yp) space.
    :param projection: Projection definition string (Returned by gdal.Dataset.GetProjectionRef)
    """
    driver = gdal.GetDriverByName('GTiff')
    rows, cols = data.shape
    dataset = driver.Create(fname, cols, rows, 1, data_type)
    dataset.SetGeoTransform(geo_transform)
    dataset.SetProjection(projection)
    band = dataset.GetRasterBand(1)
    # print(len(np.unique(data, return_inverse=True)))
    band.WriteArray(data)

    ct = gdal.ColorTable()
    for pixel_value in range(len(classes)+1):
        color_hex = COLORS[pixel_value]
        r = int(color_hex[1:3], 16)
        g = int(color_hex[3:5], 16)
        b = int(color_hex[5:7], 16)
        ct.SetColorEntry(pixel_value, (r, g, b, 255))
    band.SetColorTable(ct)

    metadata = {
        'TIFFTAG_COPYRIGHT': 'CC BY 4.0',
        'TIFFTAG_DOCUMENTNAME': 'classification',
        'TIFFTAG_IMAGEDESCRIPTION': 'Supervised classification.',
        'TIFFTAG_MAXSAMPLEVALUE': str(len(classes)),
        'TIFFTAG_MINSAMPLEVALUE': '0',
        'TIFFTAG_SOFTWARE': 'Python, GDAL, scikit-learn'
    }
    dataset.SetMetadata(metadata)

    dataset = None  # Close the file
    return

def classic_classifier(method, raster_data_path, train_data_path, validation_data_path, output_fname):
    try:
        #raster_dataset = gdal.Open(raster_data_path, gdal.GA_ReadOnly)
        raster_dataset = xr.open_rasterio(raster_data_path)
        
        raster_dataset = raster_dataset.to_dataset(name='image')
        geo_transform = raster_dataset.geo.transform
        proj = raster_dataset.geo.projection
        n_bands = len(raster_dataset.band)
        dim = ['x', 'y']
        rows, cols = [raster_dataset.sizes[xy] for xy in dim]
        
        df_raster = raster_dataset.to_dataframe().unstack("band")
        
    except RuntimeError as e:
        report_and_exit(str(e))
        
    """
    
    bands_data = []
    for b in range(1, raster_dataset.RasterCount+1):
        band = raster_dataset.GetRasterBand(b)
        bands_data.append(band.ReadAsArray())

    bands_data = np.dstack(bands_data)
    rows, cols, n_bands = bands_data.shape
    # A sample is a vector with all the bands data. Each pixel (independent of its position) is a
    # sample.
    n_samples = rows*cols
    """

    logger.debug("Process the training data")\

    # create a dataframe from each pixel and its label
    df_labeled_pixels = sample_vectors_to_raster(train_data_path, raster_dataset)
    X_train, y_train = dataframe_to_features(df_labeled_pixels)
    n_class = y_train.unique()
    
    print("\n\nDataFrame for X_train: \n", X_train.head())
    print("\n\nDataFrame for y_train: \n", y_train.head())
    # labeled_pixels = vectors_to_raster(shapefiles, rows, cols, geo_transform, proj)
    # Create dataframe each pixel of the image 
    X_predict = dataframe_to_features(df_raster)
    
    print("\n\nDataFrame for X_predict: \n", X_predict.head())
    
    """    
    is_train = np.nonzero(labeled_pixels)
    training_labels = labeled_pixels[is_train]
    training_samples = bands_data[is_train]

    flat_pixels = bands_data.reshape((n_samples, n_bands))
    """
    #
    # Perform classification
    #
    CLASSIFIERS = {
        # http://scikit-learn.org/dev/modules/generated/sklearn.ensemble.RandomForestClassifier.html
        'random-forest': RandomForestClassifier(n_jobs=4, n_estimators=10, class_weight='balanced'),
        # http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
        'svm': SVC(class_weight='balanced'),
        'linear-svm': SVC(kernel="linear", C=0.025),
        'svm-gamma': SVC(gamma=2, C=1),
        'nearest-neighbors': KNeighborsClassifier(3),
        'Gaussian-Process': GaussianProcessClassifier(1.0 * RBF(1.0)),
        'Decision-Tree': DecisionTreeClassifier(max_depth=5),
        'random-forest-1feature': RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
        'Neural-Net': MLPClassifier(alpha=1, max_iter=1000),
        'AdaBoost': AdaBoostClassifier(),
        'Naive-Bayes': GaussianNB(),
        'Quad-Discr': QuadraticDiscriminantAnalysis()
    }

    classifier = CLASSIFIERS[method]
    logger.debug("Train the classifier: %s", str(classifier))
    classifier.fit(X_train, y_train)

    logger.debug("Classifing...")
    result = classifier.predict(X_predict)

    # Reshape the result: split the labeled pixels into rows to create an image
    classification = result.reshape((rows, cols))
    write_geotiff(output_fname, n_class, classification, geo_transform, proj)
    logger.info("Classification created: %s", output_fname)

    #
    # Validate the results
    #
    if validation_data_path:
        logger.debug("Process the verification (testing) data")
        try:
            df_validation_pixels = sample_vectors_to_raster(validation_data_path, raster_dataset)
            X_test, y_test = dataframe_to_features(df_validation_pixels)
            # print("\n\nDataFrame for X_test: \n", X_test.head())
            # print("\n\nDataFrame for y_test: \n", y_test.head())
        except OSError.FileNotFoundError as e:
            report_and_exit(str(e))

        #verification_pixels = vectors_to_raster(shapefiles, rows, cols, geo_transform, proj)
        # for_verification = np.nonzero(verification_pixels)
        # verification_labels = verification_pixels[for_verification]
        # print("\n Array:", X_test.to_numpy(), "\n shape:", X_test.to_numpy().shape)
        # y_predicted = classification[X_test.to_numpy()]
        y_predicted = classifier.predict(X_test)

        logger.info("Confussion matrix:\n%s", str(
            metrics.confusion_matrix(y_test, y_predicted)))
        target_names = ['Class %s' % s for s in n_class]
        logger.info("Classification report:\n%s",
                    metrics.classification_report(y_test, y_predicted,
                                                  target_names=target_names))
        logger.info("Classification accuracy: %f",
                    metrics.accuracy_score(y_test, y_predicted))
        
        print("Confussion matrix:\n%s", str(
            metrics.confusion_matrix(y_test, y_predicted)))
        target_names = ['Class %s' % s for s in n_class]
        print("Classification report:\n%s",
                    metrics.classification_report(y_test, y_predicted,
                                                  target_names=target_names))
        print("Classification accuracy: %f",
                    metrics.accuracy_score(y_test, y_predicted))
        return y_test, y_predicted

In [79]:
classic_classifier("random-forest", raster_data_path, train_data_path, validation_data_path, str(output_fname))

/opt/conda/envs/xgeo/lib/python3.7/site-packages/xgeo/raster.py:95: UserWarning: There is no time_dim dimension in the DataArray. It will be added to the dataarray.
  "There is no {0} dimension in the DataArray. It will be added to the dataarray.".format(dim)




DataFrame for X_train: 
 band     1     2     3     4     5
0     56.0  23.0  17.0  64.0  61.0
1     59.0  23.0  19.0  75.0  54.0
2     58.0  25.0  22.0  82.0  66.0
3     60.0  24.0  19.0  75.0  61.0
4     58.0  24.0  19.0  88.0  57.0


DataFrame for y_train: 
 0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
Name: class_id, dtype: float64


DataFrame for X_predict: 
 band             1             2             3             4             5
0    -3.400000e+38 -3.400000e+38 -3.400000e+38 -3.400000e+38 -3.400000e+38
1    -3.400000e+38 -3.400000e+38 -3.400000e+38 -3.400000e+38 -3.400000e+38
2    -3.400000e+38 -3.400000e+38 -3.400000e+38 -3.400000e+38 -3.400000e+38
3    -3.400000e+38 -3.400000e+38 -3.400000e+38 -3.400000e+38 -3.400000e+38
4    -3.400000e+38 -3.400000e+38 -3.400000e+38 -3.400000e+38 -3.400000e+38
Confussion matrix:
%s [[5373    1    0]
 [   0 3977    0]
 [   1    0 6754]]
Classification report:
%s               precision    recall  f1-score   support

   Class 1.0       1.00

(0        1.0
 1        1.0
 2        1.0
 3        1.0
 4        1.0
 5        1.0
 6        1.0
 7        1.0
 8        1.0
 9        1.0
 10       1.0
 11       1.0
 12       1.0
 13       1.0
 14       1.0
 15       1.0
 16       1.0
 17       1.0
 18       1.0
 19       1.0
 20       1.0
 21       1.0
 22       1.0
 23       1.0
 24       1.0
 25       1.0
 26       1.0
 27       1.0
 28       1.0
 29       1.0
         ... 
 16076    3.0
 16077    3.0
 16078    3.0
 16079    3.0
 16080    3.0
 16081    3.0
 16082    3.0
 16083    3.0
 16084    3.0
 16085    3.0
 16086    3.0
 16087    3.0
 16088    3.0
 16089    3.0
 16090    3.0
 16091    3.0
 16092    3.0
 16093    3.0
 16094    3.0
 16095    3.0
 16096    3.0
 16097    3.0
 16098    3.0
 16099    3.0
 16100    3.0
 16101    3.0
 16102    3.0
 16103    3.0
 16104    3.0
 16105    3.0
 Name: class_id, Length: 16106, dtype: float64,
 array([1., 1., 1., ..., 3., 3., 3.]))

In [40]:
    try:
        #raster_dataset = gdal.Open(raster_data_path, gdal.GA_ReadOnly)
        raster_dataset = xr.open_rasterio(raster_data_path)
 
        raster_dataset = raster_dataset.to_dataset(name='image')
        geo_transform = raster_dataset.geo.transform
        proj = raster_dataset.geo.projection
        n_bands = len(raster_dataset.band)
        
        df_raster = raster_dataset.to_dataframe().unstack("band")
        
    except RuntimeError as e:
        report_and_exit(str(e))

/opt/conda/envs/xgeo/lib/python3.7/site-packages/xgeo/raster.py:95: UserWarning: There is no time_dim dimension in the DataArray. It will be added to the dataarray.
  "There is no {0} dimension in the DataArray. It will be added to the dataarray.".format(dim)


In [39]:
raster_dataset = xr.open_rasterio(raster_data_path)
raster_dataset.sizes["x"]

2212

In [54]:
y_train

NameError: name 'y_train' is not defined

In [49]:
raster_dataset.sizes["y"]

1170

In [33]:
dim = raster_dataset.dims
type(dim)

xarray.core.utils.Frozen

In [ ]:
features = raster_dataset.to_dataframe().unstack("band")
features

In [ ]:
raster_dataset = xr.open_rasterio(raster_data_path)

raster_dataset = raster_dataset.to_dataset(name='image')
df_raster = raster_dataset.to_dataframe().unstack("band")
df_raster